In [ ]:
import nltk
import gensim
import pandas as pd

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from collections import Counter

nltk.download("stopwords")

In [ ]:
df = pd.read_csv('lenta.csv') # dataset import

In [ ]:
tags = df["tags"].tolist() # exctract tags of news and count them
counts = Counter(tags)
print(len(counts))

In [ ]:
# https://towardsdatascience.com/nlp-extracting-the-main-topics-from-your-dataset-using-lda-in-minutes-21486f5aa925
# https://github.com/priya-dwivedi/Deep-Learning/blob/master/topic_modeling/LDA_Newsgroup.ipynb

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian") # import list of stopwords

#Preprocess function
def preprocess_text(text):
    '''
    This function split sentences and words (tokenization), get lemmas of words, delete stopwords and punctiation.
    Return list of lemmas.
    '''
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    return tokens

In [ ]:
processed_docs = []
for text in df.text[:1000]:
    processed_docs.append(preprocess_text(text))

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs) # create dictionary of words

# create "bag of words" - https://en.wikipedia.org/wiki/Bag-of-words_model
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [ ]:
# train model with 95 topics (like number of tags in news)
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 95, 
                                   id2word = dictionary,                                    
                                   passes = 10,
                                   workers = 2)

In [ ]:
# print words for every topic

for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

Я не особо знаю как лучше задавать количество топиков, но думаю, можно найти информацию по этому вопросу.
После создания модели надо будет потестировать ее вживую и посмотреть на результаты.

Вот неплохой туториал вроде бы. Его должно хватить — https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/